In [4]:
import haystack

In [2]:
print(haystack.__version__)

2.13.1


In [3]:
from haystack import Document

In [5]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

In [9]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore

In [10]:
pip show haystack-ai


Name: haystack-ai
Version: 2.13.1
Summary: LLM framework to build customizable, production-ready LLM applications. Connect components (models, vector DBs, file converters) to pipelines or agents that can interact with your data.
Home-page: https://github.com/deepset-ai/haystack
Author: 
Author-email: "deepset.ai" <malte.pietsch@deepset.ai>
License-Expression: Apache-2.0
Location: c:\Users\anjan\OneDrive\Desktop\project\myenv\Lib\site-packages
Requires: haystack-experimental, jinja2, jsonschema, lazy-imports, more-itertools, networkx, numpy, openai, posthog, pydantic, python-dateutil, pyyaml, requests, tenacity, tqdm, typing-extensions
Required-by: haystack-experimental, ollama-haystack
Note: you may need to restart the kernel to use updated packages.


In [5]:
print(dir(haystack))

['Answer', 'AsyncPipeline', 'ComponentError', 'DeserializationError', 'Document', 'ExtractedAnswer', 'GeneratedAnswer', 'Pipeline', 'PredefinedPipeline', 'SuperComponent', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'component', 'core', 'dataclasses', 'default_from_dict', 'default_to_dict', 'haystack', 'lazy_imports', 'logging', 'marshal', 'super_component', 'telemetry', 'tracing', 'utils', 'version']


In [10]:
from haystack.components.builders import PromptBuilder

In [12]:
from haystack import Document, Pipeline

In [20]:
from haystack_integrations.components.generators.ollama import OllamaGenerator


In [13]:
from datasets import load_dataset

c:\Users\anjan\OneDrive\Desktop\project\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
import pandas as pd
import os

In [ ]:
"""dataset = pd.read_csv(r"C:\Users\anjan\OneDrive\Desktop\project\myenv\songs_2000_2020_50k.csv")"""

In [21]:
dataset = pd.read_csv(r"C:\Users\anjan\OneDrive\Desktop\project\myenv\share\man\anime-dataset-2023.csv")

In [ ]:
"""
docs = [Document(content=row["Synopsis"],
                meta ={"Name":row["Name"],
                        "Genres":row["Genres"],
                        "Episodes":row["Episodes"],
                        "Rating":row["Rating"],
                        "Year":row["Year"],
                        "Studio":row["Studio"],
                        "Type":row["Type"],
                        "Status":row["Status"],
                        "Source":row["Source"],
                        "Duration":row["Duration"],
                        "Image URL":row["Image URL"],
                        "anime_id":row["anime_id"],
                        "English name":row["English name"],
                        "Other name":row["Other name"],
                        "Score":row["Score"],
                        }) for index, row in dataset.iterrows()]
"""
"""
anime_id,Name,English name,Other name,Score,
Genres,Synopsis,Type,Episodes,Aired,
Premiered,Status,Producers,Licensors,Studios,Source,Duration
,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
"""

In [27]:
docs = [
    Document(
        content=(
            f"Synopsis: {row['Synopsis']}\n"
            f"Name: {row['Name']}\n"
            f"Genres: {row['Genres']}\n"
            f"Episodes: {row['Episodes']}\n"
            f"Rating: {row['Rating']}\n"
            f"Aired: {row['Aired']}\n"
            f"Premiered: {row['Premiered']}\n"
            f"Rank: {row['Rank']}\n"
            f"Studios: {row['Studios']}\n"
            f"Type: {row['Type']}\n"
            f"Status: {row['Status']}\n"
            f"Source: {row['Source']}\n"
            f"Duration: {row['Duration']}\n"
            f"Image URL: {row['Image URL']}\n"
            f"English name: {row['English name']}\n"
            f"Other name: {row['Other name']}\n"
            f"Score: {row['Score']}\n"
        ),
        meta={
            "Name": row["Name"],
            "Genres": row["Genres"],
            "Episodes": row["Episodes"],
            "Rating": row["Rating"],
            "Studios": row["Studios"],
            "Type": row["Type"],
            "Status": row["Status"],
            "Source": row["Source"],
            "Duration": row["Duration"],
            "Image URL": row["Image URL"],
            "anime_id": row["anime_id"],
            "English name": row["English name"],
            "Other name": row["Other name"],
            "Score": row["Score"],
        }
    )
    for _, row in dataset.iterrows()
]


In [28]:
document_store = InMemoryDocumentStore()
document_store.write_documents(docs)
retriever = InMemoryBM25Retriever(document_store=document_store)

In [69]:
template = """
You are an anime recommendation system. Given the user's query, recommend the top 5 anime titles based on the provided dataset.
Content:
{% for doc in documents %}
{{ doc.content }}
{% endfor %}

Question: {{question}}
Answer:

"""

In [64]:
from haystack.components.builders import PromptBuilder

In [82]:
template =  """
Given the user's query, recommend the anime details.
Content:
{% for doc in documents %}
{{ doc.content }}
{% endfor %}

Question: {{question}}
Answer:"""

In [ ]:
#prompt_template = 
"""
Given these documents, answer the question.
Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
Question: {{ query }}
Answer:
"""

In [ ]:
prompt_builder = PromptBuilder(template=template,
    required_variables=["docs", "Questions"],)

In [34]:
generator = OllamaGenerator(
    model="mistral",
    url="http://localhost:11434/api/generate",
    generation_kwargs={
        "num_predict": 100,
        "temperature": 0.9,})

In [84]:
#main pipeline
retriever = InMemoryBM25Retriever(document_store=document_store)

prompt_builder = PromptBuilder(template=template )

generator = OllamaGenerator(
    model="mistral",
    url="http://localhost:11434",
    generation_kwargs={
        "num_predict": 100,
        "temperature": 0.9,
    }
)


rag_pipeline = Pipeline()
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", generator)
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")


Question = "Recommend me some anime titles based on the dataset."

result = rag_pipeline.run({
    "retriever": {"query": Question},
    
})

print(result["llm"]["replies"][0])


PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


1. "Bokutachi no Godzilla" (Our Godzilla) is a Japanese animated television series that premiered in 2017. The series follows the adventures of a young boy named Shunya, who finds an egg left behind by Godzilla and hatches it into a baby kaiju named G-1.

2. "Chuunibyou demo Koi ga Shitai! Movie: Take On Me" is a Japanese romantic


In [61]:
"""
rag_pipeline_b = Pipeline()
rag_pipeline_b.add_component("retriever",retriever)
rag_pipeline_b.add_component("generator",generator)
rag_pipeline_b.add_component("prompt_builder")

rag_pipeline_b.connect("retriever", "prompt_builder.documents")
rag_pipeline_b.connect("prompt_builder", "generator")
"""

SyntaxError: incomplete input (2152439363.py, line 8)

In [55]:
from dotenv import load_dotenv
import os

load_dotenv()

False

c:\Users\anjan\OneDrive\Desktop\project\myenv\Lib\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
